In [11]:
from impedance.models.circuits import CustomCircuit
from impedance.models.circuits.elements import element
import numpy as np
import scipy
import matplotlib.pyplot as plt
from impedance.visualization import plot_nyquist

In [12]:
def add_noise(z,err):
    #err is the relative SD from mean value
    z_with_noise=np.random.normal(np.real(z),err*np.abs(np.real(z)))+1j*np.random.normal(np.imag(z),err*np.abs(np.imag(z)))
    return z_with_noise

In [13]:
@element(num_params=6, units=["Ohm", "Ohm", "Ohm", "F","",""],overwrite=True)
def TLMthreeNew(p, f):
    omega = 2 * np.pi * np.array(f)
    R1, R2, R3, Q, delta1, delta2 = p[0], p[1], p[2],p[3], p[4], p[5]
    delta3=1-delta1-delta2
    k12=R2/R1
    k13=R3/R1
    Rion1=R1
    Rion2=R2
    Rion3=R3
    CTotal1=Q
    CTotal2=Q
    CTotal3=Q
    wc1=1/(Rion1*CTotal1)
    wc2=1/(Rion2*CTotal2)
    wc3=1/(Rion3*CTotal3)
    S1=np.sqrt(omega*1j/wc1)
    S2=np.sqrt(omega*1j/wc2)
    S3=np.sqrt(omega*1j/wc3)
    C1=1/np.tanh(delta1*S1)
    C2=1/np.tanh(delta2*S2)
    C3=1/np.tanh(delta3*S3)

    Z=2*(R1)*((S1*S3*C1*k12**2+S2*C2*(S1*C1*C3*k13+S3)*k12+S2**2*C3*k13)/(S1*(S3*S1*k12**2+S2*C2*(S3*C1+S1*C3*k13)*k12+S2**2*C1*C3*k13)))
    
    return Z

In [14]:
c0=CustomCircuit(initial_guess=[1015.6,675.1,418.3,6.61e-4,0.12,0.17],circuit='TLMthreeNew')
freq = np.logspace(3,-3,num=50)
z0 = c0.predict(freq)
z1=add_noise(z0,0.05)

c:\Users\Gaming\AppData\Local\Programs\Python\Python311\Lib\site-packages\impedance\models\circuits\circuits.py:145: UserWarning: Simulating circuit based on initial parameters
  warnings.warn("Simulating circuit based on initial parameters")


In [15]:
cg=CustomCircuit(initial_guess=[583.68,389.12,194.56,6,0.12,0.17],circuit='TLMthreeNew')
fit1=cg.fit(freq,z1,weight_by_modulus=False,bounds=(0,[np.inf,np.inf,np.inf,np.inf,1,1]))
zfit1=fit1.predict(freq)

C:\Users\Gaming\AppData\Local\Temp\ipykernel_12280\119171478.py:20: RuntimeWarning: overflow encountered in tanh
  C1=1/np.tanh(delta1*S1)
C:\Users\Gaming\AppData\Local\Temp\ipykernel_12280\119171478.py:21: RuntimeWarning: overflow encountered in tanh
  C2=1/np.tanh(delta2*S2)
C:\Users\Gaming\AppData\Local\Temp\ipykernel_12280\119171478.py:22: RuntimeWarning: overflow encountered in tanh
  C3=1/np.tanh(delta3*S3)
